In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import sys
from os.path import join
import time
import os

from datetime import datetime as dt
import datetime
import pytz
utc = pytz.utc
from glob import glob

In [ ]:
!pwd

In [ ]:
#!ls

In [ ]:
!ls Recordings/path/to/recording/folder/

In [ ]:
PREFIX = "recording/folder/name" #recording/name
IMAGE_PATH = join("folder/containing/PREFIX", PREFIX) 
DATA = join(IMAGE_PATH, PREFIX + '_tracker.json')
NUMBER_IMAGES = len(glob(join(IMAGE_PATH, '*.png')))

In [ ]:
NUMBER_IMAGES

In [ ]:
df = pd.read_json(DATA)

In [ ]:
DATA

In [ ]:
IMAGE_PATH

In [ ]:
#ls Recordings/2020-07-02-12-07-24-378789/

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

## Explore times

In [ ]:
ffmpeg_start_time = dt(*tuple(int(x) for x in PREFIX.split("-")), tzinfo=utc)
ffmpeg_start_time = ffmpeg_start_time + datetime.timedelta(hours=-2)

In [ ]:
ffmpeg_start_time

In [ ]:
# test
ffmpeg_start_time.microsecond

In [ ]:
# total time of JSON data
df["timestamp"].max() - df["timestamp"].min()

In [ ]:
# JSON file starts 40 seconds earlier that ffmpeg recording and stops later
df["timestamp"] - ffmpeg_start_time

In [ ]:
s = abs(df["timestamp"] - ffmpeg_start_time)
s.idxmin()

In [ ]:
s.loc[s.idxmin()]

In [ ]:
df.loc[s.idxmin()]

## Modularize and merge

In [ ]:
def find_elem_with_closest_ts(json_df, video_start):
    """Find index of row whose timestamp is closest to the video starting time"""
    s = abs(json_df["timestamp"] - video_start)
    return s.idxmin()

In [ ]:
def merge(json_df): 
    image_file_names = [PREFIX + '_' + str(i).zfill(3) + '.png' for i in json_df.index]
    merged = {image_file_names[j]:json_df.loc[j,"objects"] for j in range(len(image_file_names))}
    return merged

In [ ]:
starting_index = find_elem_with_closest_ts(df, ffmpeg_start_time)

In [ ]:
starting_index

In [ ]:
df.loc[starting_index:].reset_index()

In [ ]:
merged = merge(df.loc[starting_index:].reset_index())

In [ ]:
merged

## Draw some boxes for testing

In [ ]:
# objects found in iamge number 1
#objects_1 = merged['2020-07-02-12-07-24-378789_000.png']
objects_1 = merged['2020-07-23-11-31-26-908767_001.png']
filter_list = ['car','traffic light', 'person']
detects = [item for item in objects_1 if item['name'] in filter_list]

#cars_1 = [item for item in objects_1 if item['name']=='car']
#person_1 = [item for item in objects_1 if item['name']=='person']


In [ ]:
len(detects)

#len(cars_1)

In [ ]:
detects

In [ ]:
# Draw Bounding boxes
# Set bounding boxes path to same directory as images. 
path_to_images = os.getcwd()+ "/" + IMAGE_PATH
print(path_to_images)
for file_name in os.listdir(path_to_images):
    if file_name.endswith(".png"):
        print(file_name)
        source_img = Image.open(os.path.join(path_to_images, file_name))
        draw = ImageDraw.Draw(source_img)
        for item in detects:
            x = item["x"]
            y = item["y"]
            w = item["w"]
            h = item["h"]
            name = item["name"]
            draw.rectangle(((x-w/2, y-h/2), (x+w/2, y+h/2)), fill=None, outline="red")
            draw.text((x-w/2, y-h/2), name, font=ImageFont.truetype('Pillow/Tests/fonts/Arial.ttf'), fill="red")
        source_img.save('boxes_'+ file_name.split(".")[0] + '.jpg')

   
                                
#(glob(join(IMAGE_PATH, '*.png')))

In [ ]:
!ls

In [ ]:
#!rm boxes_2020-06-04-15-40-21-286683_*

In [ ]:
!ls Recordings/

In [ ]:
#Open any image with boxes
Image.open("boxes_.jpg")

##Done
1. add text for name parameter

## pending
2. not only cars, all objects
3. all directories
4. store images with boxes, e.g. boxes_<original_image_name>.png (so "boxes" as a prefix for the image name)
5. throw everything into a python script